The implementations of all item recommendations are done using Cornac. 
- MostPop is a non-personalized model where the most popular/rated items are recommended to everyone.

- The user and item KNN are neighborhood models used for top-n recommendations.

- NCF (Neural Collaborative Filtering) is a generalization of the matrix factorization problem using a multilayer perceptron.

- BPR (Bayesian Personalised Ranking) is esentially a Matrix Factorization algorithm which is optimized with a Bayes Criterion (BPR-OPT) in order to make the recommendation list ranking as personalized to the user (hence as "correct") as possible. 

In [1]:
import numpy as np
import pandas as pd
import cornac
from cornac.eval_methods import CrossValidation, RatioSplit
from cornac.data import Reader
from cornac.data import Dataset
from cornac.hyperopt import Discrete, GridSearch
from tabulate import tabulate

## Read and load the data using Cornac

In [2]:
# Init cornac reader object
reader = Reader() # this binarises the data (turns it into implicit feedback)

# Read both the datasets from the files using cornac
movielens_data = reader.read(fpath="../data/u.data", sep="\t")
pda_data = reader.read(fpath="../data/train-PDA2018.csv", sep=",", skip_lines=1)
print("Movielens")
print(movielens_data[:5])
print()
print("PDA")
print(pda_data[:5])

Movielens
[('196', '242', 3.0), ('186', '302', 3.0), ('22', '377', 1.0), ('244', '51', 2.0), ('166', '346', 1.0)]

PDA
[('5', '648', 5.0), ('5', '1394', 5.0), ('5', '3534', 5.0), ('5', '104', 4.0), ('5', '2735', 5.0)]


## Define models

In [3]:
# Most Popular model
most_pop_model = cornac.models.MostPop()

# KNN Models
user_knn = cornac.models.UserKNN(
    k=80, similarity="pearson", name="UserKNN"
)

item_knn = cornac.models.ItemKNN(
    k=80, similarity="pearson", name="UserKNN"
)

# NCF model: We'll use the Pre-Trained NeuMF model, since it performs better as shown in https://arxiv.org/pdf/1708.05031.pdf
gmf = cornac.models.GMF(
    num_factors=8,
    num_epochs=10,
    learner="adam",
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
mlp = cornac.models.MLP(
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_epochs=5,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
ncf_model = cornac.models.NeuMF(
    name="NeuMF_pretrained",
    learner="adam",
    num_epochs=5,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
    num_factors=gmf.num_factors,
    layers=mlp.layers,
    act_fn=mlp.act_fn,
).pretrain(gmf, mlp)

# BPR Model
bpr_model = cornac.models.BPR(
    k=10,
    learning_rate=0.01,
    lambda_reg=0.01
)

# Eval Metrics
pre_5 = cornac.metrics.Precision(k=5)
pre_10 = cornac.metrics.Precision(k=10)
rec_5 = cornac.metrics.Recall(k=5)
rec_10 = cornac.metrics.Recall(k=10)
ndcg = cornac.metrics.NDCG()
auc = cornac.metrics.AUC()

## Experiment Variables

In [4]:
results_table = [] # List that will contain the results
cv_n_folds = 5 # define number of k splits for cross validation
rating_threshold = 3 # This parameter is the threshold used for ranking metrics

# Algorithms we will be using
cornac_algorithms = {
    "MostPop": most_pop_model,
    "UserKNN": user_knn,
    "ItemKNN": item_knn,
    "BPR": bpr_model, 
    #"NCF": ncf_model
}
# Datasets
datasets = {
    "ML100": movielens_data,
    "PDA2018": pda_data
}

## 5-Fold Cross Validation

In [5]:
for dataset in datasets.keys():
    for algorithm in cornac_algorithms.keys():
        print("Running 5-fold cross validation with", algorithm, "on", dataset, "dataset ...\n\n")
        # Define Cornac cross validation object
        cv = CrossValidation(
            data=datasets[dataset],
            n_folds=cv_n_folds,
            rating_threshold=rating_threshold, # This parameter is the threshold used for ranking metrics
            seed = 0,
            verbose=True
        )
        # Define Cornac experiment (put everything together)
        experiment = cornac.Experiment(
            eval_method=cv,
            models=[cornac_algorithms[algorithm]],    
            metrics=[pre_5, pre_10, rec_5, rec_10, auc, ndcg],
        )
        experiment.run()
        for entry in experiment.result:
            results_dict = entry[0].metric_avg_results
            new_line = [dataset+"-"+algorithm, results_dict['Precision@5'], results_dict['Precision@10'], \
                        results_dict['Recall@5'], results_dict['Recall@10'], results_dict['NDCG@-1']]
            results_table.append(new_line)

Running 5-fold cross validation with MostPop on ML100 dataset ...


rating_threshold = 3.0
exclude_unknowns = True
Fold: 1
---
Training data:
Number of users = 943
Number of items = 1648
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1382
Number of ratings = 19966
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 943
Number of items = 1382
Number of ratings = 19966
---
Total users = 943
Total items = 1648

[MostPop] Training started!

[MostPop] Evaluation started!



Fold: 2
---
Training data:
Number of users = 943
Number of items = 1652
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 942
Number of items = 1371
Number of ratings = 19967
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 942
Number of items = 1371
Number of ratings = 19967
---
Total users = 943
Total items = 1652

[MostPop] Training started!

[MostPop] Evaluation started!



Fold: 3
---
Training data:
Number of users = 943
Number of items = 1651
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 940
Number of items = 1390
Number of ratings = 19965
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 940
Number of items = 1390
Number of ratings = 19965
---
Total users = 943
Total items = 1651

[MostPop] Training started!

[MostPop] Evaluation started!



Fold: 4
---
Training data:
Number of users = 943
Number of items = 1656
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1397
Number of ratings = 19969
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 943
Number of items = 1397
Number of ratings = 19969
---
Total users = 943
Total items = 1656

[MostPop] Training started!

[MostPop] Evaluation started!



Fold: 5
---
Training data:
Number of users = 943
Number of items = 1646
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 942
Number of items = 1383
Number of ratings = 19959
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 942
Number of items = 1383
Number of ratings = 19959
---
Total users = 943
Total items = 1646

[MostPop] Training started!

[MostPop] Evaluation started!


...


Fold: 2
---
Training data:
Number of users = 5685
Number of items = 1821
Number of ratings = 376569
Max rating = 5.0
Min rating = 1.0
Global mean = 3.6
---
Test data:
Number of users = 5294
Number of items = 1791
Number of ratings = 94127
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5294
Number of items = 1791
Number of ratings = 94127
---
Total users = 5685
Total items = 1821

[BPR] Training started!

[BPR] Evaluation started!



Fold: 3
---
Training data:
Number of users = 5684
Number of items = 1823
Number of ratings = 376568
Max rating = 5.0
Min rating = 1.0
Global mean = 3.6
---
Test data:
Number of users = 5328
Number of items = 1798
Number of ratings = 94129
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5328
Number of items = 1798
Number of ratings = 94129
---
Total users = 5684
Total items = 1823

[BPR] Training started!

[BPR] Evaluation started!



Fold: 4
---
Training data:
Number of users = 5684
Number of items = 1822
Number of ratings = 376569
Max rating = 5.0
Min rating = 1.0
Global mean = 3.6
---
Test data:
Number of users = 5303
Number of items = 1787
Number of ratings = 94125
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5303
Number of items = 1787
Number of ratings = 94125
---
Total users = 5684
Total items = 1822

[BPR] Training started!

[BPR] Evaluation started!



Fold: 5
---
Training data:
Number of users = 5682
Number of items = 1820
Number of ratings = 376569
Max rating = 5.0
Min rating = 1.0
Global mean = 3.6
---
Test data:
Number of users = 5281
Number of items = 1799
Number of ratings = 94121
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5281
Number of items = 1799
Number of ratings = 94121
---
Total users = 5682
Total items = 1820

[BPR] Training started!

[BPR] Evaluation started!




TEST:
...
[BPR]
       |    AUC | NDCG@-1 | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
------ + ------ + ------- + ------------ + ----------- + --------- + -------- + --------- + --------
Fold 0 | 0.9031 |  0.3904 |       0.0882 |      0.0962 |    0.1009 |   0.0570 |    3.3421 |   7.3767
Fold 1 | 0.9029 |  0.3912 |       0.0898 |      0.0954 |    0.0979 |   0.0536 |    4.4609 |   7.1950
Fold 2 | 0.9023 |  0.3885 |       0.0889 |      0.0931 |    0.0939 |   0.0514 |    4.9192 |   7.0318
Fold 3 | 0.8996 |  0.3874 |       0.0876 |      0.0938 |    0.0959 |   0.0514 |    3.8431 |   7.5820
Fold 4 | 0.9000 |  0.3885 |       0.0889 |      0.0943 |    0.0944 |   0.0536 |    3.6668 |   8.2425
------ + ------ + ------- + ------------ + ----------- + --------- + -------- + --------- + --------
Mean   | 0.9016 |  0.3892 |       0.0887 |      0.0946 |    0.0966 |   0.0534 |    4.0464 |   7.4856
Std    | 0.0015 |  0.0014 |       0.0007 |      0.0011 |    0.0026 |   0.

In [6]:
# Display results of running the algorithms
results_table_headers = ["Recommender", "Pre@5", "Pre@10", "Rec@5", "Rec@10", "NDCG"]
print(tabulate(results_table, results_table_headers, tablefmt="pipe"))

| Recommender     |       Pre@5 |     Pre@10 |       Rec@5 |      Rec@10 |     NDCG |
|:----------------|------------:|-----------:|------------:|------------:|---------:|
| ML100-MostPop   | 0.0945802   | 0.0886291  | 0.0431486   | 0.0799196   | 0.405632 |
| ML100-UserKNN   | 0.00021254  | 0.00053135 | 0.000265675 | 0.000401886 | 0.285699 |
| ML100-ItemKNN   | 0.000850159 | 0.0010627  | 0.00071437  | 0.00178445  | 0.27017  |
| ML100-BPR       | 0.120298    | 0.113177   | 0.0634099   | 0.118859    | 0.45788  |
| PDA2018-MostPop | 0.0739311   | 0.0701476  | 0.0394338   | 0.0697588   | 0.348685 |
| PDA2018-UserKNN | 0.000264851 | 0.00145668 | 0.000414723 | 0.00311618  | 0.262592 |
| PDA2018-ItemKNN | 0.00469164  | 0.00548619 | 0.00149846  | 0.00333021  | 0.247353 |
| PDA2018-BPR     | 0.0962164   | 0.0881763  | 0.0569755   | 0.10091     | 0.390439 |


In [8]:
# Export data
# Export the results to a csv file
results_df = pd.DataFrame(results_table, columns=["Recommender", "Pre@5", "Pre@10", "Rec@5", "Rec@10", "NDCG"])
results_df.to_csv("../data/cornac_item_recommendation_results.csv")